In [10]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [11]:
train_err  = pd.read_csv('LG/err_group_train_target.csv')
train_err

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver,time
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,20201101025616,20201130212033,10000,model_3,05.15.2138,NaN
1,0.0,0.0,0.0,0.0,9.0,1.0,1.0,0.0,0.0,0.0,...,0.0,74.0,28.0,0.0,20201101020415,20201112030039,10001,model_2,04.33.1185,2.020111e+13
2,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,28.0,1.0,20201112030617,20201130222646,10001,model_2,04.33.1261,2.020111e+13
3,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,20201101030251,20201130211528,10002,model_3,05.15.2138,NaN
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,20201101182532,20201110072130,10003,model_2,04.33.1185,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24841,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,38.0,0.0,0.0,20201101071624,20201119031243,24997,model_0,04.22.1750,2.020112e+13
24842,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,20.0,8.0,5.0,20201119031831,20201130231434,24997,model_0,04.22.1778,2.020112e+13
24843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20201101050758,20201103050737,24998,model_0,04.22.1750,2.020111e+13
24844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,20201103051329,20201130210420,24998,model_0,04.22.1778,2.020111e+13


In [12]:
train_err['time']=train_err['time'].fillna(0)

In [13]:
train_err

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver,time
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,20201101025616,20201130212033,10000,model_3,05.15.2138,0.000000e+00
1,0.0,0.0,0.0,0.0,9.0,1.0,1.0,0.0,0.0,0.0,...,0.0,74.0,28.0,0.0,20201101020415,20201112030039,10001,model_2,04.33.1185,2.020111e+13
2,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,28.0,1.0,20201112030617,20201130222646,10001,model_2,04.33.1261,2.020111e+13
3,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,20201101030251,20201130211528,10002,model_3,05.15.2138,0.000000e+00
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,20201101182532,20201110072130,10003,model_2,04.33.1185,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24841,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,38.0,0.0,0.0,20201101071624,20201119031243,24997,model_0,04.22.1750,2.020112e+13
24842,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,20.0,8.0,5.0,20201119031831,20201130231434,24997,model_0,04.22.1778,2.020112e+13
24843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20201101050758,20201103050737,24998,model_0,04.22.1750,2.020111e+13
24844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,20201103051329,20201130210420,24998,model_0,04.22.1778,2.020111e+13


In [14]:
train_err_=train_err.copy()
for x in range(24846):
    if train_err_['time_min'][x]<train_err_['time'][x]<train_err_['time_max'][x]:
        train_err_['time'][x]=1
    else:
        train_err_['time'][x]=0

In [15]:
train_err_

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver,time
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,20201101025616,20201130212033,10000,model_3,05.15.2138,0.0
1,0.0,0.0,0.0,0.0,9.0,1.0,1.0,0.0,0.0,0.0,...,0.0,74.0,28.0,0.0,20201101020415,20201112030039,10001,model_2,04.33.1185,0.0
2,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,28.0,1.0,20201112030617,20201130222646,10001,model_2,04.33.1261,1.0
3,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,20201101030251,20201130211528,10002,model_3,05.15.2138,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,20201101182532,20201110072130,10003,model_2,04.33.1185,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24841,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,38.0,0.0,0.0,20201101071624,20201119031243,24997,model_0,04.22.1750,1.0
24842,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,20.0,8.0,5.0,20201119031831,20201130231434,24997,model_0,04.22.1778,0.0
24843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20201101050758,20201103050737,24998,model_0,04.22.1750,0.0
24844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,20201103051329,20201130210420,24998,model_0,04.22.1778,1.0


In [84]:
T=train_err_.groupby(['user_id', 'fwver'])['time'].sum()

In [85]:
T1=list(T)
for x in range(24062):
    if T1[x]>0:
        T1[x]=1

In [86]:
T2=pd.DataFrame(T1)

In [87]:
train_err_dup=train_err_.drop_duplicates(['user_id','model_nm','fwver'])

In [88]:
train_err_dup.index=np.arange(24062)
train_err_dup

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver,time
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,20201101025616,20201130212033,10000,model_3,05.15.2138,0.0
1,0.0,0.0,0.0,0.0,9.0,1.0,1.0,0.0,0.0,0.0,...,0.0,74.0,28.0,0.0,20201101020415,20201112030039,10001,model_2,04.33.1185,0.0
2,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,28.0,1.0,20201112030617,20201130222646,10001,model_2,04.33.1261,1.0
3,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,20201101030251,20201130211528,10002,model_3,05.15.2138,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,20201101182532,20201110072130,10003,model_2,04.33.1185,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24057,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,38.0,0.0,0.0,20201101071624,20201119031243,24997,model_0,04.22.1750,1.0
24058,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,20.0,8.0,5.0,20201119031831,20201130231434,24997,model_0,04.22.1778,0.0
24059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20201101050758,20201103050737,24998,model_0,04.22.1750,0.0
24060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,20201103051329,20201130210420,24998,model_0,04.22.1778,1.0


In [89]:
test_err= pd.read_csv('LG/err_group_test.csv')
test_err

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver
0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,46.0,0.0,0.0,20201101030227,20201127024946,30000,model_1,04.16.3553
1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,42.0,0.0,1.0,20201118073855,20201122050431,30000,model_1,04.16.3571
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,1.0,20201127094708,20201130192105,30000,model_2,04.33.1261
3,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20201101042135,20201130221906,30001,model_3,05.15.2138
4,0.0,0.0,0.0,0.0,31.0,2.0,1.0,0.0,0.0,0.0,...,0.0,14.0,95.0,1.0,0.0,20201101003616,20201126040300,30002,model_0,04.22.1750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,156.0,1.0,0.0,20201101003242,20201119052538,44996,model_0,04.22.1750
24092,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,173.0,1.0,3.0,20201119053129,20201130220212,44996,model_0,04.22.1778
24093,0.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,...,3.0,0.0,28.0,44.0,0.0,20201101000032,20201127035735,44997,model_0,04.22.1750
24094,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,5.0,0.0,20201127040356,20201130232034,44997,model_0,04.22.1778


### Test Data의 Index에 결측치가 존재하여 수정 필요

In [101]:
test_err2 = test_err
test_err2.shape

(24096, 46)

In [147]:
additional_index = pd.DataFrame(np.zeros(len(test_err2.columns)).reshape(1, -1))
columns_names = test_err2.columns
additional_index.columns = columns_names
additional_index['model_nm'] = test_err2.model_nm.mode()
additional_index.user_id = [43262]   # 리스트로 입력 안 하면 오류 뜬다.
additional_index.index = [43262]
additional_index

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver
43262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43262,model_0,0.0


In [148]:
final_test_data = test_err2.append(additional_index)
final_test_data

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver
0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,46.0,0.0,0.0,2.020110e+13,2.020113e+13,30000,model_1,04.16.3553
1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,42.0,0.0,1.0,2.020112e+13,2.020112e+13,30000,model_1,04.16.3571
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,1.0,2.020113e+13,2.020113e+13,30000,model_2,04.33.1261
3,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.020110e+13,2.020113e+13,30001,model_3,05.15.2138
4,0.0,0.0,0.0,0.0,31.0,2.0,1.0,0.0,0.0,0.0,...,0.0,14.0,95.0,1.0,0.0,2.020110e+13,2.020113e+13,30002,model_0,04.22.1750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24092,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,173.0,1.0,3.0,2.020112e+13,2.020113e+13,44996,model_0,04.22.1778
24093,0.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,...,3.0,0.0,28.0,44.0,0.0,2.020110e+13,2.020113e+13,44997,model_0,04.22.1750
24094,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,5.0,0.0,2.020113e+13,2.020113e+13,44997,model_0,04.22.1778
24095,0.0,0.0,0.0,2.0,14.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,269.0,0.0,0.0,2.020110e+13,2.020113e+13,44998,model_1,04.16.3553


In [149]:
final_test_data_dup=final_test_data.drop_duplicates(['user_id','model_nm','fwver'])
final_test_data_dup

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver
0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,46.0,0.0,0.0,2.020110e+13,2.020113e+13,30000,model_1,04.16.3553
1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,42.0,0.0,1.0,2.020112e+13,2.020112e+13,30000,model_1,04.16.3571
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,1.0,2.020113e+13,2.020113e+13,30000,model_2,04.33.1261
3,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.020110e+13,2.020113e+13,30001,model_3,05.15.2138
4,0.0,0.0,0.0,0.0,31.0,2.0,1.0,0.0,0.0,0.0,...,0.0,14.0,95.0,1.0,0.0,2.020110e+13,2.020113e+13,30002,model_0,04.22.1750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24092,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,173.0,1.0,3.0,2.020112e+13,2.020113e+13,44996,model_0,04.22.1778
24093,0.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,...,3.0,0.0,28.0,44.0,0.0,2.020110e+13,2.020113e+13,44997,model_0,04.22.1750
24094,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,5.0,0.0,2.020113e+13,2.020113e+13,44997,model_0,04.22.1778
24095,0.0,0.0,0.0,2.0,14.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,269.0,0.0,0.0,2.020110e+13,2.020113e+13,44998,model_1,04.16.3553


In [150]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt #시각화 패키지
import seaborn as sns #시각화 패키지
plt.style.use('fivethirtyeight') # 테마바꾸기
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from tqdm import tnrange, tqdm_notebook, notebook
import time

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, scale

from sklearn.linear_model import RidgeCV, Ridge, LassoCV, Lasso, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
import lightgbm as lgb

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans

# 실루엣 분석 metric 값을 구하기 위한 API 추가
from sklearn.metrics import silhouette_samples, silhouette_score

from sklearn.datasets import make_blobs, load_boston
import matplotlib.cm as cm

In [151]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = train_err_dup.drop(["time","time_min","time_max","user_id","fwver"],axis=1)
train_y = train_err_dup.time

In [152]:
test_x = final_test_data_dup.drop(["time_min","time_max","user_id","fwver"],axis=1)

# Modeling

In [153]:
print(train_err_dup.model_nm.unique())
print(test_x.model_nm.unique())

['model_3' 'model_2' 'model_0' 'model_1' 'model_7' 'model_4' 'model_5'
 'model_8' 'model_6']
['model_1' 'model_2' 'model_3' 'model_0' 'model_4' 'model_5' 'model_6'
 'model_7' 'model_8']


In [154]:
from sklearn.preprocessing import LabelEncoder

encoder_model_tr1.fit(train_x['model_nm'])
label_model_tr1=encoder_model_tr1.transform(train_x['model_nm'])
train_x['model_nm']=label_model_tr1

label_model_tr2=encoder_model_tr1.transform(test_x['model_nm'])
test_x['model_nm']=label_model_tr2

In [155]:
train_x.shape

(24062, 42)

In [156]:
test_x.shape

(24097, 42)

In [182]:
#run traning
LGBM=LGBMClassifier(boosting_type='gbdt',
                    objective='binary', 
                    metric='auc', 
                    random_state=0, 
                    num_boost_round = 1000)
#result=cross_val_score(LGBM,train_features, train_target,cv=10,scoring='roc_auc')
#print('The cross validated score for LightBoost is:',result.mean())

In [183]:
LGBM.fit(train_x, train_y)

LGBMClassifier(metric='auc', num_boost_round=300, objective='binary',
               random_state=0)

In [184]:
proba=pd.DataFrame(final_test_data['user_id'])
proba['problem']=LGBM.predict_proba(test_x)[:,1]
#LGBM=LGBM.drop([0,1],axis=1)
#LGBM.to_csv(r"/content/LGBM.csv", index=False)

In [185]:
proba

,user_id,problem
0,30000,0.207543
1,30000,0.155332
2,30000,0.278045
3,30001,0.243787
4,30002,0.189341
...,...,...
24092,44996,0.109974
24093,44997,0.872143
24094,44997,0.040590
24095,44998,0.487701


In [187]:
last2=proba.groupby(['user_id']).max()

In [179]:
last

,problem
user_id,
30000,0.278045
30001,0.243787
30002,0.189341
30003,0.632162
30004,0.189423
...,...
44994,0.115567
44995,0.363322
44996,0.566527


In [189]:
last2.to_csv(r"LG_LGBM_5.csv", index=True)